In [26]:
import pandas as pd

import sys
sys.path.append("../src/")

from src import CSV_DELIMITER
from src.fingerprint import Fingerprint

In [35]:
ATTRIBUTES_TO_EXCLUDE = [Fingerprint.COUNTER, Fingerprint.CREATION_TIME, Fingerprint.END_TIME]

In [16]:
df = pd.read_csv("./data/processed/fingerprint_dataset.csv", delimiter=CSV_DELIMITER, engine="python")

In [ ]:
attributes_tp_analyze = [attribute for attribute in df.columns.tolist() if attribute not in ATTRIBUTES_TO_EXCLUDE]

for attribute in attributes_tp_analyze:
    nb_distinct_values = df[attribute].value_counts().count()
    print("{} : {:d} distinct values".format(attribute, nb_distinct_values))

id : 35 distinct values
addressHttp : 35 distinct values
userAgentHttp : 35 distinct values
acceptHttp : 35 distinct values
connectionHttp : 35 distinct values
encodingHttp : 35 distinct values
languageHttp : 35 distinct values
orderHttp : 35 distinct values
pluginsJSHashed : 35 distinct values
platformJS : 35 distinct values
cookiesJS : 35 distinct values
dntJS : 35 distinct values
timezoneJS : 35 distinct values
resolutionJS : 35 distinct values
localJS : 35 distinct values
sessionJS : 35 distinct values
canvasJSHashed : 35 distinct values
fontsFlashHashed : 35 distinct values
resolutionFlash : 35 distinct values
languageFlash : 35 distinct values
platformFlash : 35 distinct values
